In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import datetime

import PIL.Image as Image
import tensorflow as tf
import tensorflow_hub as hub
%load_ext tensorboard

In [2]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SIZE = (160,160)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

# Freeze the base_model
base_model.trainable = False

9420800/9406464 [==============================] - 13s 1us/step


In [3]:
# Let's take a look at the base model architecture
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 80, 80, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [16]:
#Loading the data and prefetch it

batch_size = 32
img_height = 160
img_width = 160

data_root = tf.keras.utils.get_file(
  'flower_photos',
  'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)


class_names = np.array(train_ds.class_names)
print(class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.
['daisy' 'dandelion' 'roses' 'sunflowers' 'tulips']


In [14]:
#Create data augmentation pipeline

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [17]:
# tf.keras.applications.MobileNetV2 use as the base model.
# This model expects pixel values in [-1, 1], but at this point, the pixel values in your images are in [0, 255]
# To rescale them, use the preprocessing method included with the model.
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input


In [6]:
image_batch, label_batch = next(iter(train_ds))
feature_batch = base_model(image_batch)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 1280)


In [13]:
prediction_layer = tf.keras.layers.Dense(len(class_names))
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 5)


In [18]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [25]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [19]:
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0     

In [20]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.

In [26]:
NUM_EPOCHS = 10

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

Epoch 1/10
92/92 [==============================] - 85s 880ms/step - loss: 1.7395 - accuracy: 0.3089 - val_loss: 1.3562 - val_accuracy: 0.4482
Epoch 2/10
92/92 [==============================] - 69s 747ms/step - loss: 1.2753 - accuracy: 0.5010 - val_loss: 1.0552 - val_accuracy: 0.5995
Epoch 3/10
92/92 [==============================] - 64s 693ms/step - loss: 1.0139 - accuracy: 0.6131 - val_loss: 0.8726 - val_accuracy: 0.7030
Epoch 4/10
92/92 [==============================] - 73s 792ms/step - loss: 0.8653 - accuracy: 0.6744 - val_loss: 0.7631 - val_accuracy: 0.7548
Epoch 5/10
92/92 [==============================] - 65s 703ms/step - loss: 0.7620 - accuracy: 0.7214 - val_loss: 0.6893 - val_accuracy: 0.7738
Epoch 6/10
92/92 [==============================] - 71s 777ms/step - loss: 0.6873 - accuracy: 0.7510 - val_loss: 0.6337 - val_accuracy: 0.7847
Epoch 7/10
92/92 [==============================] - 67s 725ms/step - loss: 0.6605 - accuracy: 0.7575 - val_loss: 0.5910 - val_accuracy: 0.7997

## Fine Tunning

In [27]:
base_model.trainable = True


In [28]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

Number of layers in the base model:  154


In [29]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])

In [30]:
fine_tune_epochs = 10
total_epochs =  NUM_EPOCHS + fine_tune_epochs

history_fine = model.fit(train_ds,
                        validation_data=val_ds,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         callbacks=tensorboard_callback)

Epoch 10/20
92/92 [==============================] - 120s 1s/step - loss: 0.4579 - accuracy: 0.8300 - val_loss: 0.3970 - val_accuracy: 0.8583
Epoch 11/20
92/92 [==============================] - 110s 1s/step - loss: 0.3674 - accuracy: 0.8641 - val_loss: 0.3610 - val_accuracy: 0.8692
Epoch 12/20
92/92 [==============================] - 104s 1s/step - loss: 0.3299 - accuracy: 0.8774 - val_loss: 0.3249 - val_accuracy: 0.8774
Epoch 13/20
92/92 [==============================] - 105s 1s/step - loss: 0.2773 - accuracy: 0.8958 - val_loss: 0.3340 - val_accuracy: 0.8719
Epoch 14/20
92/92 [==============================] - 105s 1s/step - loss: 0.2665 - accuracy: 0.8965 - val_loss: 0.3035 - val_accuracy: 0.8828
Epoch 15/20
92/92 [==============================] - 104s 1s/step - loss: 0.2351 - accuracy: 0.9094 - val_loss: 0.2870 - val_accuracy: 0.8869
Epoch 16/20
92/92 [==============================] - 103s 1s/step - loss: 0.2183 - accuracy: 0.9217 - val_loss: 0.3141 - val_accuracy: 0.8828
Epoch 